In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import itertools
import re
pd.set_option('display.max_colwidth', -1)

In [9]:
# convert Time column to Time in secs and save to new column
def convert_time_to_secs(x):
    m,s = map(int, x.split(':'))
    return m*60+s

In [10]:
# Scrape UFC249 data
# event_url = "https://en.wikipedia.org/wiki/UFC_249"
# event_date = "May 9, 2020"
# event_name = "UFC 249 Ferguson vs. Gaethje"

# Scrape UFC Fight Night: Smith vs. Teixeira data
event_url = "https://en.wikipedia.org/wiki/UFC_Fight_Night:_Smith_vs._Teixeira"
event_date = "May 13, 2020"
event_name = "UFC Fight Night Smith vs. Teixeira"

url_request = requests.get(event_url).text
soup = BeautifulSoup(url_request, 'html.parser')
data = []
table = soup.find('table',{'class':'toccolours'})
table_rows = table.find_all('tr')
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')]) 
df = pd.DataFrame(data, columns=['Weight class', 'Fighter1', 'Result', 'Fighter2', 'Method', 'Round', 'Time','Note'])
df = df[~df['Weight class'].isnull()]
df.insert(0, "Date", event_date) 
df['Date']= pd.to_datetime(df['Date'],format= '%B %d, %Y')
# df['Round']= 0
# df['Time']= 0
df.insert(1, "Event", event_name)
df.insert(9,"Time (secs)", "0")
df['Time (secs)'] = df['Time'].apply(convert_time_to_secs)
# df.fillna("-", inplace=True)
df.replace("","-", inplace=True)
df.to_csv("data/"+ event_name + ".csv", index=False)

In [11]:
df

,Date,Event,Weight class,Fighter1,Result,Fighter2,Method,Round,Time,Time (secs),Note
2,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Light Heavyweight,Glover Teixeira,def.,Anthony Smith,TKO (punches),5,1:04,64,-
3,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Heavyweight,Ben Rothwell,def.,Ovince Saint Preux,"Decision (split) (28–29, 29–28, 29–28)",3,5:00,300,-
4,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Lightweight,Drew Dober,def.,Alexander Hernandez,TKO (punches),2,4:25,265,-
5,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Bantamweight,Ricky Simon,def.,Ray Borg,"Decision (split) (28–29, 29–28, 29–28)",3,5:00,300,-
6,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Heavyweight,Andrei Arlovski,def.,Philipe Lins,"Decision (unanimous) (30–27, 30–27, 29–28)",3,5:00,300,-
7,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Lightweight,Thiago Moisés,def.,Michael Johnson,Submission (Ankle Lock),2,0:25,25,-
9,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Women's Bantamweight,Sijara Eubanks,def.,Sarah Moras,"Decision (unanimous) (30–27, 30–27, 30–26)",3,5:00,300,-
10,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Lightweight,Omar Morales,def.,Gabriel Benítez,"Decision (unanimous) (30–27, 29–28, 29–28)",3,5:00,300,-
11,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Featherweight,Brian Kelleher,def.,Hunter Azure,KO (punches),2,3:40,220,-
12,2020-05-13,UFC Fight Night: Smith vs. Teixeira,Heavyweight,Chase Sherman,def.,Isaac Villanueva,TKO (elbow and punches),2,0:49,49,-


In [71]:
# Collect event poster
posters_df = pd.read_csv("data_latest/ufc_posters_may09.csv")
posters_df = posters_df.drop(columns=['id'])

In [72]:
# event_url = "https://en.wikipedia.org/wiki/UFC_249"
# event_date = "May 9, 2020"
# event_name = "UFC 249 Ferguson vs. Gaethje"
event_url = "https://en.wikipedia.org/wiki/UFC_Fight_Night:_Smith_vs._Teixeira"
event_date = "May 13, 2020"
event_name = "UFC Fight Night Smith vs. Teixeira"

In [73]:
url_request = requests.get(event_url).text
soup = BeautifulSoup(url_request, 'html.parser')
images = soup.find_all('img')
for image in images:
    image_link = "http:" + image['src']
    print(image_link) 

http://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Merge-arrow.svg/50px-Merge-arrow.svg.png
http://upload.wikimedia.org/wikipedia/en/thumb/d/de/UFC_Fight_Night-_Smith_v_Teixeira.jpeg/220px-UFC_Fight_Night-_Smith_v_Teixeira.jpeg
http://upload.wikimedia.org/wikipedia/en/thumb/4/48/Folder_Hexagonal_Icon.svg/16px-Folder_Hexagonal_Icon.svg.png
http://upload.wikimedia.org/wikipedia/commons/thumb/3/37/People_icon.svg/16px-People_icon.svg.png
http://upload.wikimedia.org/wikipedia/en/thumb/4/4a/Commons-logo.svg/12px-Commons-logo.svg.png
http://en.wikipedia.org/wiki/Special:CentralAutoLogin/start?type=1x1
http:/static/images/wikimedia-button.png
http:/static/images/poweredby_mediawiki_88x31.png


In [74]:
# image = "http://upload.wikimedia.org/wikipedia/en/thumb/6/61/Verified_UFC_249_Poster.jpg/220px-Verified_UFC_249_Poster.jpg"
image = "http://upload.wikimedia.org/wikipedia/en/thumb/d/de/UFC_Fight_Night-_Smith_v_Teixeira.jpeg/220px-UFC_Fight_Night-_Smith_v_Teixeira.jpeg"

In [75]:
df = pd.DataFrame({'Date': [event_date], 'Event': event_name, \
                   "wikipage": event_url, "poster_url": image})
df['Date']= pd.to_datetime(df['Date'],format= '%b %d, %Y')
df

,Date,Event,wikipage,poster_url
0,2020-05-13,UFC Fight Night Smith vs. Teixeira,https://en.wikipedia.org/wiki/UFC_Fight_Night:_Smith_vs._Teixeira,http://upload.wikimedia.org/wikipedia/en/thumb/d/de/UFC_Fight_Night-_Smith_v_Teixeira.jpeg/220px-UFC_Fight_Night-_Smith_v_Teixeira.jpeg


In [76]:
posters_df = posters_df.append(df, ignore_index=True, sort=True)

In [77]:
# Writer posters_df to file
posters_df.to_csv("data_latest/ufc_posters_may13.csv", index=True)